In [ ]:
import requests                         # to make an API request and get recent data
import pandas as pd                     # for data
import numpy as np                      # for math
from datetime import date, timedelta    # for calculating future dates
from prophet import Prophet             # stores the model
from matplotlib import pyplot as plt    # for graphing forecast

# class for invalid API Key exception
class APIKeyException(Exception): 
    def __init__(self, message):
        self.message = message  # custom error message for many types of errors (api key not found, api key invalid, rate limited, etc)

In [ ]:
# ENTER YOUR API KEY HERE!!!
APIKEY = ""
FUTUREFORECASTDAYS = 20

In [ ]:
# gets symbol
symbol = input("Enter the stock symbol you want to forecast: ")

# parameters 
url = "https://www.alphavantage.co/query"
params = {
    "function": "TIME_SERIES_DAILY_ADJUSTED",
    "apikey": APIKEY,
    "symbol": symbol,
    "datatype": "json",
    "outputsize": "compact"
}

# get data
data = requests.get(url, params=params).json()

# try to look at data otherwise api key is limited
try:
    data = data["Time Series (Daily)"]
except KeyError:
    try: 
        data["Error Message"]       # checks if there is an error message
        raise APIKeyException("Your API key is invalid.")
    except KeyError:                         # otherwise it must be a rate limit
        raise APIKeyException("You have been rate limited. Try again later.")

# gets values
closePrices = [float(day["5. adjusted close"]) for day in list(data.values())[::-1]]     # 100 most recent
dates = [date for date in list(data.keys())[::-1]]

# creates dataframe
df = pd.DataFrame({
    'ds': dates,
    'y': closePrices
})

# predicts next 10 days
model = Prophet()
model.fit(df)

# gets the date ten days in the future 
future = model.make_future_dataframe(periods=FUTUREFORECASTDAYS)
forecast = model.predict(future)
model.plot(forecast)
plt.show()